In [36]:
import os
import pymongo
import pandas as pd
from datetime import datetime as dt
from src.features.dedup_model import load_results
from src.data.utils import load_json_as_df, create_dir
from src.data.image_utils import download_images_from_group
import src.models.nima_model as nima 



In [12]:
from src.constants import (
    RAW_DIR_PATH,
    INTERIM_DIR_PATH,
    PROCESSED_DIR_PATH
)

In [2]:
DB_USERNAME = os.getenv("MONGO_USERNAME")
DB_PASSWORD = os.getenv("MONGO_PASSWORD")
DB_URL = os.getenv("MONGO_URL")
DB_NAME = os.getenv("MONGO_DB")
DB_NAME

'mlops-project'

In [6]:
client = pymongo.MongoClient(
    f"mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority"
)
client

MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True)

In [7]:
db = client[DB_NAME]
db.head

Collection(Database(MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True), 'mlops-project'), 'head')

In [8]:
db.list_collection_names()


['duplicated_images_model_results']

In [9]:
collectionDuplicated = db['duplicated_images_model_results']
df = pd.DataFrame(collectionDuplicated.find())

In [31]:
df.head()

,_id,date,group_id,base_image_id,duplicated_image_id,probability
0,63c530068113af969cb993ec,2023-01-16 11:07:50.773,102292696432481,1631,1703,0.916148
1,63c530068113af969cb993ed,2023-01-16 11:07:50.774,102292696432481,1631,1710,0.868106
2,63c530068113af969cb993ee,2023-01-16 11:07:50.774,102292696432481,1631,1741,0.868106
3,63c530068113af969cb993ef,2023-01-16 11:07:50.774,102292696432481,1631,1744,0.868106
4,63c530068113af969cb993f0,2023-01-16 11:07:50.774,102292696432481,1631,1760,1.000000


In [35]:
df1 = df[df['group_id']=='2224086878288']
df1

,_id,date,group_id,base_image_id,duplicated_image_id,probability
16876,63c5303d8113af969cb9d5d8,2023-01-16 11:08:45.547,2224086878288,1,108,0.895422
16877,63c5303d8113af969cb9d5d9,2023-01-16 11:08:45.547,2224086878288,1,119,1.000000
16878,63c5303d8113af969cb9d5da,2023-01-16 11:08:45.547,2224086878288,1,127,1.000000
16879,63c5303d8113af969cb9d5db,2023-01-16 11:08:45.547,2224086878288,1,145,0.895422
16880,63c5303d8113af969cb9d5dc,2023-01-16 11:08:45.547,2224086878288,1,176,0.895422
...,...,...,...,...,...,...
18661,63c5303d8113af969cb9dcd1,2023-01-16 11:08:45.548,2224086878288,98,6,1.000000
18662,63c5303d8113af969cb9dcd2,2023-01-16 11:08:45.548,2224086878288,99,105,1.000000
18663,63c5303d8113af969cb9dcd3,2023-01-16 11:08:45.548,2224086878288,99,110,1.000000
18664,63c5303d8113af969cb9dcd4,2023-01-16 11:08:45.548,2224086878288,99,33,1.000000


In [13]:
images_json_file_name = "images.json"
images_json_file_path = os.path.join(RAW_DIR_PATH, images_json_file_name)

In [14]:
images_df = load_json_as_df(images_json_file_path, index_col="image_id")
images_df.head()

,group_id,image_url
image_id,,
1,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
2,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
3,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
4,2224086878288,https://i.travelapi.com/hotels/57000000/569800...
5,2224086878288,https://i.travelapi.com/hotels/57000000/569800...


In [22]:
images_dir_path = os.path.join(INTERIM_DIR_PATH, "images")
create_dir(images_dir_path)

In [23]:
for group_id, df_group in df.groupby('group_id'):

    lista = df_group['base_image_id'].to_list() +df_group['duplicated_image_id'].to_list()
    mylist = list(dict.fromkeys(lista))

    df_images_group= pd.DataFrame (mylist, columns = ['image_id'])
    df_images_group['group_id'] = group_id

    merged_group = pd.merge(left=df_images_group, right=images_df, left_on=['group_id','image_id'], right_on=['group_id','image_id'])

    download_images_from_group(images_df, group_id, (images_dir_path ))
    



365it [00:00, 5663.34it/s]
215it [00:00, 4188.36it/s]
25it [00:00, 822.92it/s]
125it [00:00, 3162.27it/s]
132it [00:00, 3904.18it/s]
40it [00:00, 2230.24it/s]
5it [00:00, 499.56it/s]
230it [00:00, 5108.07it/s]
210it [00:00, 2684.31it/s]
330it [00:00, 5506.99it/s]
225it [00:00, 5278.45it/s]
170it [00:00, 3937.03it/s]
345it [00:00, 6691.12it/s]
120it [00:00, 2877.36it/s]


In [24]:
model= "MobileNet"
src = os.path.join('/app','src')
features = os.path.join(src,'features')
nima_model_folder = os.path.join(features,'nima_model')
weights = os.path.join(nima_model_folder,'weights_mobilenet_aesthetic_0.07.hdf5')



In [25]:
images_dir_path

'/app/data/interim/images'

In [37]:
for group_id, df_group in df.groupby('group_id'):
    image_folder = os.path.join(images_dir_path,group_id)
    nima.predictImages(model,weights,image_folder)

/app/src/models/nima_model/__init__.py:27: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  return model.predict_generator(data_generator, workers=8, use_multiprocessing=True, verbose=1)
/usr/local/lib/python3.10/site-packages/keras/engine/data_adapter.py:925: RuntimeWarning: overflow encountered in cast
  return np.array(t, dtype=backend.floatx())


KeyboardInterrupt: 

In [14]:
type(images_df)

pandas.core.frame.DataFrame

In [15]:
lista = dfs[0]['base_image_id'].to_list() +dfs[0]['duplicated_image_id'].to_list()

type(lista)

list

In [35]:
merged_inner = pd.merge(left=df_images_group, right=images_df, left_on=['group_id','image_id'], right_on=['group_id','image_id'])

In [39]:
df_images_group


,image_id,group_id
0,1631,102292696432481
1,1632,102292696432481
2,1633,102292696432481
3,1634,102292696432481
4,1635,102292696432481
...,...,...
360,1959,102292696432481
361,1993,102292696432481
362,1927,102292696432481
363,1961,102292696432481


In [26]:
lista_merged = list(dict.fromkeys(merged_inner['group_id']))
lista_merged

df2 = merged_inner.pivot_table(index = ['image_id'], aggfunc ='size')
df2



image_id
1631    1
1632    1
1633    1
1634    1
1635    1
       ..
1991    1
1992    1
1993    1
1994    1
1995    1
Length: 365, dtype: int64